# Utilities

## Classes

In [ ]:
import re
import random
import math

# Objects of class CNF represent formulas in Conjunctive Normal Form.
# The attribute clauses is a list of object of class Clause.
class CNF:
  def __init__(self, clauses):
    self.clauses = clauses
  
  def setClauses(self, clauses):
    self.clauses = clauses
  
  def addClause(self, clause):
    self.clauses.append(clause)

  def getClauses(self):
    return self.clauses
  
  def printCNF(self):
    i = 0
    for C in self.getClauses():
      i += 1
      j = 0
      print("Clause ", i)
      for x in C.getLiterals():
        j += 1
        print("Literal ", j)
        print(x.name, x.polarity)

# Objects of class Clause represent disjunctions.
# The attribute literals is a list of object of class Literal.
class Clause:
  def __init__(self, literals):
    self.literals = literals
  
  def setLiterals(self, literals):
    self.literals = literals
  
  def addLiteral(self, lit):
    self.literals.append(lit)

  def getLiterals(self):
    return self.literals
  
  def equalTo(self, C):
    if len(self.literals) != len(C.literals):
      return False
    
    for x1 in self.literals:
      is_present = False
      for x2 in C.literals:
        if x1.equalTo(x2):
          is_present = True
      if not is_present:
        return False
    
    return True

# Objects of class Literal represent a variable and its polarity.
# If the polarity is True, the literal is non-negated, otherwise it is negated.
class Literal:
  def __init__(self, name, polarity):
    self.name = name
    self.polarity = polarity
  
  def getName(self):
    return self.name
  
  def getPolarity(self):
    return self.polarity

  def getValue(self, I):  # getValue returns the truth value of the literal given an interpretation
    return ((not I.getAssignment().get(self.name) or self.polarity) and (not self.polarity or I.getAssignment().get(self.name)))

  def equalTo(self, lit):
    return self.name == lit.name and self.polarity == lit.polarity

  def oppositeOf(self, lit):
    return self.name == lit.name and self.polarity == (not lit.polarity)

# The attribute assignment is a dict: keys are variables and values are truth assignment to related variables.
# The attribute fixed is a list of variables for which we already have the best assignment and it must be not modified.
class Assignment:
  def __init__(self, assignment, fixed):
    self.assignment = assignment
    self.fixed = fixed
  
  def getFixedVars(self):
    return self.fixed
  
  def addFixedVar(self, var):
    self.fixed.append(var)
  
  def setFixedVars(self, fixed_vars):
    self.fixed = fixed_vars

  def flipAssignment(self, var):
    self.assignment.update({var: (not self.assignment.get(var))})
  
  def modifyAssignment(self, var, val):
    self.assignment.update({var: val})
  
  def setAssignment(self, assignment):
    self.assignment = assignment

  def getAssignment(self):
    return self.assignment
  
  def serialize(self):
    string = ''.join(map(str, map(int, self.assignment.values())))
    return string

  def deserialize(self, string):
    i = 0
    for var in self.assignment.keys():
      self.assignment.update({var: bool(int(string[i]))})
      i += 1

## Functions

In [ ]:
# It generates a random assignment for all the variables but those in fixed
def randomInterpretation(I):
  for var in I.getAssignment().keys():
    if var not in I.getFixedVars():
      I.getAssignment().update({var: random.choice([True, False])})

# It checks if a clause is satisfied under an interpretation (at least one literal must be True)
def isClauseSatisfied(C, I):
  for x in C.getLiterals():
    if x.getValue(I):
      return True
  return False

# It checks if a clause is satisfied under an interpretation restricted to those variables that are fixed
def isClauseSatisfiedByFixedVars(C, I):
  for x in C.getLiterals():
    if x.getValue(I) and x.getName() in I.getFixedVars():
      return True
  return False

# It checks if a clause is unsatisfiable under an interpretation restricted to those variables that are fixed
def isClauseUnsatisfiableByFixedVars(C, I):
  for x in C.getLiterals():
    if x.getName() not in I.getFixedVars():
      return False
    elif x.getValue(I):
      return False
  return True

# It counts the number of satisfied clauses under an interpretation
def numberSatisfiedClauses(cnf, I):
  res = 0
  for C in cnf.getClauses():
    if isClauseSatisfied(C, I):
      res += 1
  return res

# It counts the number of satisfied clauses under an interpretation restricted to those variables that are fixed
def numberSatisfiedClausesByFixedVars(cnf, I):
  res = 0
  for C in cnf.getClauses():
    if isClauseSatisfiedByFixedVars(C, I):
      res += 1
  return res

# It counts the number of unsatisfiable clauses under an interpretation restricted to those variables that are fixed
def numberUnsatisfiableClausesByFixedVars(cnf, I):
  res = 0
  for C in cnf.getClauses():
    if isClauseUnsatisfiableByFixedVars(C, I):
      res += 1
  return res

# It checks if a formula is satisfied under an interpretation (all clauses must be True)
def isFormulaSatisfied(cnf, I):
  return numberSatisfiedClauses(cnf, I) == len(cnf.getClauses())

# It chooses a random clause among the unsatisfied ones in a formula
def pickUnsatisfiedClause(cnf, I):
  unsatisfiedClauses = []

  for C in cnf.getClauses():
    if not isClauseSatisfied(C, I):
      unsatisfiedClauses.append(C)
  
  return random.choice(unsatisfiedClauses)

# It checks how many clauses are satisfied after flipping a variable and performs the best flip (ties are broken randomly).
# The list of variables to use is a param, when unspecified we consider all the variables (in both cases, we filter out fixed ones).
def bestFlip(cnf, I, vars=[]):
  max_sat = numberSatisfiedClauses(cnf, I)
  best_var = []
  
  if vars == []:
    vars = I.getAssignment().keys()
  
  vars = list(filter(lambda var: var not in I.getFixedVars(), vars))
  
  for var in vars:
    I.flipAssignment(var)
    num_sat = numberSatisfiedClauses(cnf, I)

    if num_sat > max_sat:
      max_sat = num_sat
      best_var = [var]
    elif num_sat == max_sat:
      best_var.append(var)
    
    I.flipAssignment(var)

  if best_var != []:
    flip_var = random.choice(best_var)
    I.flipAssignment(flip_var)

# Creating a SAT problem instance

  ## Prefix Notation

In [ ]:
symbols = ['¬', '∧', '↑', '∨', '↓', '→', '↔', '⊕']
bin_symbols = ['∧', '↑', '∨', '↓', '→', '↔', '⊕']
un_symbols = ['¬']

# It homologates different representations for the operators in a single notation.
def uniformOperatorNotation(string):
  string = re.sub('nor', '↓', string)
  string = re.sub('nand', '↑', string)
  string = re.sub('iff|if and only if|<->|<=>', '↔', string)
  string = re.sub('implies|->|=>', '→', string)
  string = re.sub('xor|\^', '⊕', string)
  string = re.sub('and|\*|&', '∧', string)
  string = re.sub('or|\|\||\+|\|', '∨', string)
  string = re.sub('not|~|-', '¬', string)
  return string

# It returns False if the formula is ill-formed with respect to the syntax.
def checkInputValidity(string, stack, i):
  if (string[i] in symbols):
    if (i == 0 or string[i-1] == ')' or string[i-1] in bin_symbols):
      return False
  if (string[i] in bin_symbols and i == len(string)-1):
    return False
  if (string[i] == ')'):
    if (i == len(string)-1 or (i != 0 and string[i-1] in un_symbols)):
      return False
  if (string[i] == '('):
    if (i == 0 or ')' not in stack or string[i-1] in bin_symbols):
      return False
  return True

# It returns the priority of an operator when parentheses are not used (higher result means higher priority).
def getOperatorPriority(op):
  if (op == '¬'):
    return 5
  elif (op == '∧' or op == '↑'):
    return 4
  elif (op == '∨' or op == '↓'):
    return 3
  elif (op == '→'):
    return 2
  elif (op == '↔' or op == '⊕'):
    return 1
  return 0

# It turns the formula into prefix notation, using a stack to manage operators (with priority and parentheses) while scanning the formula.
# It return the formula as a list and a random interpretation for the variables (it returns None if something goes wrong).
def infixToPrefix(string):
  string = uniformOperatorNotation(string).replace(" ", "")[::-1]

  regex = re.compile("^[^(¬|∧|↑|∨|↓|→|↔|⊕|\(|\))]*")
  stack = []

  I = Assignment({}, [])
  res = ''
  i = 0

  if len(re.findall("\(", string)) != len(re.findall("\)", string)):
    return None

  while i < len(string):
    if not checkInputValidity(string, stack, i):
      return None

    operand = re.search(regex, string[i:]).group(0)

    if operand != '':
      I.getAssignment().update({operand[::-1]: random.choice([True, False])})
      res += operand + ' '
      i += len(operand)
    elif string[i] in symbols:
      while (stack != [] and getOperatorPriority(string[i]) < getOperatorPriority(stack[-1])):
        res += stack[-1] + ' '
        stack.pop()
      stack.append(string[i])
      i += 1
    elif string[i] == ')':
      stack.append(string[i])
      i += 1
    elif string[i] == '(':
      while (stack[-1] != ')'):
        res += stack[-1] + ' '
        stack.pop()
      stack.pop()
      i += 1
    
  for op in stack[::-1]:
    res += op + ' '
    
  return res[::-1].strip().split(' '), I

## CNF

In [ ]:
# It takes as input an operator ∧ or ∨ and two CNF formulas, and performs the operator on them returning the resulting CNF formula.
# The ∨ case deletes a literal if it is duplicated in a clause, and deletes a clause if it contains opposite literals.
def operatorHandler(operator, operand1, operand2):
  if operator == '∧':
    clauses = []
    for C1 in operand1.getClauses():
      same_clause = False
      for C2 in operand2.getClauses():
        if C1.equalTo(C2):
          same_clause = True
      if not same_clause:
        clauses.append(C1)
    clauses = clauses + operand2.getClauses()
    return CNF(clauses)
  elif operator == '∨':
    clauses = []
    for C1 in operand1.getClauses():
      for C2 in operand2.getClauses():
        clause = []
        del_clause = False
        for x1 in C1.getLiterals():
          same_lit = False
          for x2 in C2.getLiterals():
            if x1.equalTo(x2):
              same_lit = True
            if x1.oppositeOf(x2):
              del_clause = True
              break
          if not same_lit:
            clause.append(x1)
          if del_clause:
            break
        if not del_clause:
          clauses.append(Clause(clause + C2.getLiterals()))
    return CNF(clauses)

# It takes a formula expressed in prefix notation (as a list) and returns the CNF (as an object of the class CNF).
# It is a recursive function that handles each propositional operator (negated case are treated separately).
# The index returned with the CNF formula is useful at each step to discriminate where an operand ends within the list.
def convertToCNF(string, i):
  if string[i] not in symbols:
    return CNF([Clause([Literal(string[i], True)])]), i
  if string[i] == '∧' or string[i] == '∨':
    operand1, last = convertToCNF(string, i+1)
    operand2, last = convertToCNF(string, last+1)
    return operatorHandler(string[i], operand1, operand2), last
  if string[i] == '→':
    string[i] = '∨'
    string.insert(i+1, '¬')
    operand1, last = convertToCNF(string, i+1)
    operand2, last = convertToCNF(string, last+1)
    return operatorHandler(string[i], operand1, operand2), last
  if string[i] == '↔':
    string[i] = '∧'
    string.insert(i+1, '∨')
    operand1, last1 = convertToCNF(string, i+2)
    string.insert(last1+1, '¬')
    operand2, last2 = convertToCNF(string, last1+1)
    first_half = operatorHandler(string[i+1], operand1, operand2)
    string[last2+1:last2+1] = ['∨', '¬'] + string[i+2:last1+1] + string[last1+2:last2+1]
    second_half, last = convertToCNF(string, last2+1)
    return operatorHandler(string[i], first_half, second_half), last
  if string[i] == '⊕':
    string[i] = '¬'
    string.insert(i+1, '↔')
    return convertToCNF(string, i)
  if string[i] == '↑':
    string[i] = '¬'
    string.insert(i+1, '∧')
    return convertToCNF(string, i)
  if string[i] == '↓':
    string[i] = '¬'
    string.insert(i+1, '∨')
    return convertToCNF(string, i)
  if string[i] == '¬':
    if string[i+1] not in symbols:
      return CNF([Clause([Literal(string[i+1], False)])]), i+1
    if string[i+1] == '¬':
      return convertToCNF(string, i+2)
    if string[i+1] == '∧':
      string[i] = '∨'
      string[i+1] = '¬'
      operand1, last = convertToCNF(string, i+1)
      string.insert(last+1, '¬')
      operand2, last = convertToCNF(string, last+1)
      return operatorHandler(string[i], operand1, operand2), last
    if string[i+1] == '∨':
      string[i] = '∧'
      string[i+1] = '¬'
      operand1, last = convertToCNF(string, i+1)
      string.insert(last+1, '¬')
      operand2, last = convertToCNF(string, last+1)
      return operatorHandler(string[i], operand1, operand2), last
    if string[i+1] == '→':
      string[i] = '∧'
      del string[i+1]
      operand1, last = convertToCNF(string, i+1)
      string.insert(last+1, '¬')
      operand2, last = convertToCNF(string, last+1)
      return operatorHandler(string[i], operand1, operand2), last
    if string[i+1] == '↔':
      string[i] = '∧'
      string[i+1] = '∨'
      operand1, last1 = convertToCNF(string, i+2)
      operand2, last2 = convertToCNF(string, last1+1)
      first_half = operatorHandler(string[i+1], operand1, operand2)
      string[last2+1:last2+1] = ['∨', '¬'] + string[i+2:last1+1] + ['¬'] + string[last1+1:last2+1]
      second_half, last = convertToCNF(string, last2+1)
      return operatorHandler(string[i], first_half, second_half), last
    if string[i+1] == '⊕':
      string[i] = '↔'
      del string[i+1]
      return convertToCNF(string, i)
    if string[i+1] == '↑':
      string[i] = '∧'
      del string[i+1]
      return convertToCNF(string, i)
    if string[i] == '↓':
      string[i] = '∨'
      del string[i+1]
      return convertToCNF(string, i)

## Random instances

In [ ]:
# It generates random CNF formulas with n_vars (number of variables) and n_clauses (number of clauses).
# Last two parameters are used to establish how many literals a clause contain and whether this is a strict requirement.
def genRandomInstances(n_vars, n_clauses, k_lit_in_C=3, k_exact=True):
  cnf = CNF([])
  I = Assignment({}, [])

  for i in range(n_clauses):
    clause = Clause([])
    k = k_lit_in_C if k_exact else random.randint(1, k_lit_in_C)

    vars = [str(var) for var in range(1, n_vars+1)]

    for j in range(k):
      var_name = random.choice(vars)
      vars.remove(var_name)

      lit = Literal(var_name, random.choice([True,False]))
      clause.addLiteral(lit)
    
    cnf.addClause(clause)
  
  for var in range(1, n_vars+1):
    I.getAssignment().update({str(var): random.choice([True, False])})

  return cnf, I

# Unit Propagation and Pure Literals Rule

## Unit Propagation

In [ ]:
# It performs Unit Propagation on the formula until it's possible.
# If there is a clause with only a literal, the function set the assignment of that literal to satisfy the clause.
# If an assignment is performed, the variable involved is added to fixed list and the function is called recursively.
def unitPropagation(cnf, I):
  new_assignment = False
  print("Unit propagation")
  print("Fixed vars:")
  print(I.getFixedVars())

  for C in cnf.getClauses():
    lit = []
    satisfied = False

    if len(I.getFixedVars()) == len(I.getAssignment().keys()):
      break

    for x in C.getLiterals():
      if x.getName() in I.getFixedVars() and x.getValue(I):
        satisfied = True
      elif x.getName() not in I.getFixedVars():
        lit.append(x)

    if len(lit) == 1 and not satisfied:
      I.modifyAssignment(lit[0].getName(), lit[0].getPolarity())
      I.addFixedVar(lit[0].getName())
      new_assignment = True
  
  if new_assignment:
    unitPropagation(cnf, I)

## Pure Literals

In [ ]:
# It performs Pure Literals Rule on the formula until it's possible.
# First, the clauses is filtered because we won't consider the already satisfied ones (satisfied by the variables in fixed).
# We maintain a polarity_dict to check if a var appears always with the same polarity, in which case that var is assigned consistently.
# If an assignment is performed, the variable involved is added to fixed list and the function is called recursively.
def pureLiterals(cnf, I):
  print("Pure Literals")
  print("Fixed vars:")
  print(I.getFixedVars())

  polarity_dict = {}
  new_assignment = False

  clauses = cnf.getClauses()
  sat_clauses = []

  for C in clauses:
    for x in C.getLiterals():
      if x.getName() in I.getFixedVars() and x.getValue(I) and C not in sat_clauses:
        sat_clauses.append(C)
  
  clauses = list(filter(lambda C: C not in sat_clauses, clauses))

  for C in clauses:
    for x in C.getLiterals():
      if x.getName() in polarity_dict:
        if x.getPolarity() != polarity_dict.get(x.getName()):
          polarity_dict.update({x.getName(): None})
      else:
        polarity_dict.update({x.getName(): x.getPolarity()})

  for pair in polarity_dict.items():
    if len(I.getFixedVars()) == len(I.getAssignment().keys()):
      break
    
    if pair[1] != None and pair[0] not in I.getFixedVars():
      I.modifyAssignment(pair[0], pair[1])
      I.addFixedVar(pair[0])
      new_assignment = True

  if new_assignment:
    pureLiterals(cnf, I)

# SAT Algorithms

## GSAT

In [ ]:
# It implements the base variant of GSAT algorithm.
# The variables max_restart and max_flips control respectively outer and inner loops.
# The core step (greedy hill-climbing) is performed by the function bestFlip.
def GSAT(cnf, I, max_restart, max_flips):
  for i in range(max_restart):
    if i == 0:
      print("Starting GSAT...")
    else:
      print("Restarting...")

    randomInterpretation(I)

    print("Number of satisfied clauses {}".format(numberSatisfiedClauses(cnf, I)))

    if isFormulaSatisfied(cnf, I):
      return True

    for j in range(max_flips):
      bestFlip(cnf, I)

      print("Iteration {} - Number of satisfied clauses {}".format(j, numberSatisfiedClauses(cnf, I)))
      if isFormulaSatisfied(cnf, I):
        return True

  return False

## WalkSAT

In [ ]:
# It implements the base variant of WalkSAT algorithm.
# As for GSAT, the variables max_restart and max_flips control respectively outer and inner loops.
# On the contrary wrt GSAT, bestFlip is called only in some iterations (probabilistically) and only on variables from a chosen clause.
# The clause is chosen among the unsatisfied ones by the function pickUnsatisfiedClause
def WalkSAT(cnf, I, prob, max_restart, max_flips):
  for i in range(max_restart):
    if i == 0:
      print("Starting WalkSAT...")
    else:
      print("Restarting...")

    randomInterpretation(I)

    print("Number of satisfied clauses {}".format(numberSatisfiedClauses(cnf, I)))

    if isFormulaSatisfied(cnf, I):
      return True

    for j in range(max_flips):
      C = pickUnsatisfiedClause(cnf, I)

      if random.random() < prob:
        lits = [x for x in C.getLiterals() if x.getName() not in I.getFixedVars()]
        if lits != []:
          x = random.choice(lits)
          I.flipAssignment(x.getName())
      else:
        vars = [x.getName() for x in C.getLiterals()]
        bestFlip(cnf, I, vars)

      print("Iteration {} - Number of satisfied clauses {}".format(j, numberSatisfiedClauses(cnf, I)))
      if isFormulaSatisfied(cnf, I):
        return True

  return False

## SASAT

In [ ]:
# It implements the base variant of Simulated Annealing algorithm.
# The variables max_restart and min_temp control respectively outer and inner loops.
# Temperature starts by max_temp and degrades with time passing by depending on the decay rate.
# For each variable, its assignment is probabilistically flipped based on temperature and delta (increasing number of satisfied clauses).
def SASAT(cnf, I, max_restart, max_temp, min_temp, decay_rate):
  for i in range(max_restart):
    if i == 0:
      print("Starting SASAT...")
    else:
      print("Restarting ({})...".format(i))

    randomInterpretation(I)

    print("Number of satisfied clauses {}".format(numberSatisfiedClauses(cnf, I)))

    if isFormulaSatisfied(cnf, I):
      return True

    temp = max_temp
    j = 0

    while temp > min_temp:
      # Update temperature
      temp = max_temp * math.exp(-1*j*decay_rate)

      for var in I.getAssignment().keys():
        if var not in I.getFixedVars():
          # Compute for each (non-fixed) var the gain (delta)
          num_sat = numberSatisfiedClauses(cnf, I)

          I.flipAssignment(var)
          delta = numberSatisfiedClauses(cnf, I) - num_sat

          # Compute probability and hold the new interpretation according to that value
          prob = 1 / (1 + math.exp((-1*delta) / temp))
          if random.random() > prob:
            I.flipAssignment(var)
      
      print("Iteration {} - Number of satisfied clauses {}".format(j, numberSatisfiedClauses(cnf, I)))
      if isFormulaSatisfied(cnf, I):
        return True
      j += 1

  return False

# It implements a variant of Simulated Annealing algorithm with Random Walks.
# In this case, with a certain probability we use simulated annealing approach;
# Otherwise, we flip the variable if the gain is greater than zero.
def SASAT_RW(cnf, I, prob, max_restart, max_temp, min_temp, decay_rate):
  for i in range(max_restart):
    if i == 0:
      print("Starting SASAT...")
    else:
      print("Restarting ({})...".format(i))

    randomInterpretation(I)

    print("Number of satisfied clauses {}".format(numberSatisfiedClauses(cnf, I)))

    if isFormulaSatisfied(cnf, I):
      return True

    temp = max_temp
    j = 0

    while temp > min_temp:
      # Update temperature
      temp = max_temp * math.exp(-1*j*decay_rate)

      for var in I.getAssignment().keys():
        if var not in I.getFixedVars():
          # Compute for each (non-fixed) var the gain (delta)
          num_sat = numberSatisfiedClauses(cnf, I)

          I.flipAssignment(var)
          delta = numberSatisfiedClauses(cnf, I) - num_sat

          # Hold the new interpretation according to probabilities obtained by random walk and delta
          if random.random() < prob:
            if random.random() > 1 / (1 + math.exp((-1*delta) / temp)):
              I.flipAssignment(var)
          else:
            if delta < 0:
              I.flipAssignment(var)
      
      print("Iteration {} - Number of satisfied clauses {}".format(j, numberSatisfiedClauses(cnf, I)))
      if isFormulaSatisfied(cnf, I):
        return True
      j += 1

  return False

## Genetic Local Search

In [ ]:
# It implements a Genetic Algorithm with Stochastic Local Search.
# Population is a list of lists: each sublist has three components, i.e. a serialized interpretation, 
# the related number of satisfied clauses, and the cumulative percentage value of fitness function (useful for selection).
def GLS(cnf, I, num_pop, epochs, mutation_prob, elitism_prop):
  print("Starting Genetic Local Search...")

  population = []
  n = 0
  cumulative_fitness = 0.0
  
  # New population
  for i in range(num_pop):
    randomInterpretation(I)

    if isFormulaSatisfied(cnf, I):
      return True

    num_sat = numberSatisfiedClauses(cnf, I)
    population.append([I.serialize(), num_sat, cumulative_fitness])
    n += num_sat
  
  for elem in population:
    elem[2] = elem[1] / n + cumulative_fitness
    cumulative_fitness = elem[2]
  
  # Init epochs 
  for epoch in range(epochs):
    print("Epoch {}".format(epoch))
    children = []

    # Generation of new individuals
    for i in range(num_pop):
      parents = []

      # Parent selection
      for j in range(2):
        parent = ''
        
        select_prob = random.random()

        for elem in population:
          if elem[2] > select_prob:
            parent = elem[0]
            break

        parents.append(parent)

      # Crossover
      child = ''
      for gene in range(len(parents[0])):
        child += random.choice([parents[0][gene], parents[1][gene]])

      # Mutation
      mut_child = ''

      if random.random() < mutation_prob:
        for gene in range(len(child)):
          if random.random() < 0.5:
            mut_child += str(int(not bool(int(child[gene]))))
          else:
            mut_child += child[gene]
      else:
        mut_child = child

      # SLS for the new individual
      I.deserialize(mut_child)

      num_sat = numberSatisfiedClauses(cnf, I)
      improvement = True

      while improvement:
        bestFlip(cnf, I)

        if numberSatisfiedClauses(cnf, I) > num_sat:
          num_sat = numberSatisfiedClauses(cnf, I)
        else:
          improvement = False
      
      if isFormulaSatisfied(cnf, I):
        return True

      children.append([I.serialize(), num_sat])
    
    # Generational replacement with elitism
    population = sorted(population, key=lambda x:x[1], reverse=True)
    children = sorted(children, key=lambda x:x[1], reverse=True)

    print("Number of satisfied clauses {}".format(children[0][1]))
    
    del population[int(elitism_prop * num_pop):]
    del children[num_pop - int(elitism_prop * num_pop):]

    n = 0
    cumulative_fitness = 0.0
    
    for elem in population:
      n += elem[1]
    
    for elem in children:
      population.append([elem[0], elem[1], cumulative_fitness])
      n += elem[1]
    
    for elem in population:
      elem[2] = elem[1] / n + cumulative_fitness
      cumulative_fitness = elem[2]

  return False

# Max-SAT

## Johnson’s Algorithm

In [ ]:
# It implements Johnson’s Algorithm for Max-SAT problems.
# The dictionary "weights" stores a list for each clause: the first element is the actual weight of the clause;
# the second and the third elements are lists of variables that occur respectively with positive and negative polarity in that clause.
def Johnson(cnf, I):
  weights = {}

  # Creating weights dictionary
  for C in cnf.getClauses():
    pos_occ = []
    neg_occ = []
    for x in C.getLiterals():
      if x.getPolarity():
        pos_occ.append(x.getName())
      else:
        neg_occ.append(x.getName())
    
    weights.update({C: [1, pos_occ, neg_occ]})
  
  # Assignment
  for var in I.getAssignment().keys():
    pos_set = []
    neg_set = []
    pos_w = 0
    neg_w = 0

    # Splitting clauses according to the polarity of current var and summing up weights for the two groups
    for elem in weights.items():
      if var in elem[1][1]:
        pos_set.append(elem[0])
        pos_w += elem[1][0]
      elif var in elem[1][2]:
        neg_set.append(elem[0])
        neg_w += elem[1][0]
    
    # Setting the assignment and adjusting weights dictionary
    if pos_w >= neg_w:
      I.modifyAssignment(var, True)

      for pos_C in pos_set:
        del weights[pos_C]

      for neg_C in neg_set:
        l = weights.get(neg_C)
        l[0] = l[0]*2
        weights.update({neg_C: l})
    
    else:
      I.modifyAssignment(var, False)

      for neg_C in neg_set:
        del weights[neg_C]

      for pos_C in pos_set:
        l = weights.get(pos_C)
        l[0] = l[0]*2
        weights.update({pos_C: l})

## Bidirectional Bounds Refinement

In [ ]:
# It implements a Bidirectional Bounds Refinement algorithm for Max-SAT.
# In each iteration, it maintains a mean value to compare which truth value must be assigned to a var. 
def BBR(cnf, I):
  n = len(cnf.getClauses())
  mean = n / 2

  for var in I.getAssignment().keys():
    I.addFixedVar(var)

    # Compute the new mean and gain when var is True
    I.modifyAssignment(var, True)
    pos_lb = numberSatisfiedClausesByFixedVars(cnf, I)
    pos_ub = n - numberUnsatisfiableClausesByFixedVars(cnf, I)
    pos_mean = (pos_lb + pos_ub) / 2
    pos_gain = pos_mean - mean

    # Compute the new mean and gain when var is False
    I.modifyAssignment(var, False)
    neg_lb = numberSatisfiedClausesByFixedVars(cnf, I)
    neg_ub = n - numberUnsatisfiableClausesByFixedVars(cnf, I)
    neg_mean = (neg_lb + neg_ub) / 2
    neg_gain = neg_mean - mean

    mean = neg_mean
    
    # Choose between holding the "false" interpretation for var or switching to a "true" one (updating mean)
    if neg_gain < 0:
      I.modifyAssignment(var, True)
      mean = pos_mean
    elif pos_gain > 0:
      if random.random() < (pos_gain / (pos_gain + neg_gain)):
        I.modifyAssignment(var, True)
        mean = pos_mean

## Look-Ahead SLS

In [ ]:
# It implements a GSAT-like SLS with Look-Ahead mechanism.
# num_first_lvl is used to determine how many variables must be considered before the actual look-ahead flip.
def LookAhead(cnf, I, max_restart, max_flips, num_first_lvl):
  n = len(cnf.getClauses())
  best_assignment = I.getAssignment()
  max_sat = numberSatisfiedClauses(cnf, I)

  for i in range(max_restart):
    if i == 0:
      print("Starting SLS...")
    else:
      print("Restarting...")

    randomInterpretation(I)
    curr_sat = numberSatisfiedClauses(cnf, I)

    if isFormulaSatisfied(cnf, I):
      return n

    for j in range(max_flips):
      # Perform the best flip
      bestFlip(cnf, I)
      num_sat = numberSatisfiedClauses(cnf, I)

      if num_sat > curr_sat:
        curr_sat = num_sat
      else:
        # If best flip does not increase the number of satisfied clauses, try to look-ahead
        curr_assignment = I.getAssignment()
        first_lvl_lit = []

        # Select a set of "first level" literals
        k = 0
        while len(first_lvl_lit) < num_first_lvl and k < 2*num_first_lvl:
          C = pickUnsatisfiedClause(cnf, I)
          lit = random.choice(C.getLiterals())
          
          if lit not in first_lvl_lit:
            first_lvl_lit.append(lit)
          
          k += 1
        
        # Flip a "first level" literal and perform the best flip.
        for x in first_lvl_lit:
          I.flipAssignment(x.getName())

          bestFlip(cnf, I)
          num_sat = numberSatisfiedClauses(cnf, I)

          if num_sat > curr_sat:
            curr_sat = num_sat
            break
          else:
            I.setAssignment(curr_assignment)

      print("Iteration {} - Number of satisfied clauses {}".format(j, curr_sat))
      if isFormulaSatisfied(cnf, I):
        return n

    if curr_sat > max_sat:
      best_assignment = I.getAssignment().copy()
      max_sat = curr_sat

  I.setAssignment(best_assignment)
  return max_sat

# Model Counting

## ExactCount

In [ ]:
# It counts the number of models for a given formula.
def exactCount(cnf, I):
  vars = [v for v in I.getAssignment().keys() if v not in I.getFixedVars()]
  n_vars = len(vars)
  count = 0

  for var in vars:
    I.modifyAssignment(var, False)
  
  # Simulate the generation of the "rows" present in the truth table of the formula
  prec = 0
  for k in range(2**len(vars)):
    for idx, switch in enumerate(format(k ^ prec, "b")):
      if switch == '1':
        I.flipAssignment(vars[n_vars-1 - idx])
    
    prec = k

    if isFormulaSatisfied(cnf, I):
      count += 1
  
  return count

## ApproxCount

In [ ]:
# It implements ApproxCount for the Model Counting problem.
# The multiplier computation is based on at most k solutions (k control the quality of the approximation).
def ApproxCount(cnf, I, k):
  multiplier = 1

  n_left_vars = len([v for v in I.getAssignment().keys() if v not in I.getFixedVars()])

  # While the formula is not feasible for ExactCount
  while n_left_vars > 15:
    i = 0
    samples = []
    
    # Sample at most k solutions (satisfying interpretations)
    while len(samples) < k and i < 2*k:
      if random.random() < 0.5:
        sol = WalkSAT(cnf, I, 0.25, 100, 100)
      else:
        sol = SASAT(cnf, I, 100, 1.0, 0.1, 0.05)

      if sol and I.getAssignment() not in samples:
        samples.append(I.getAssignment().copy())
      
      i += 1
    
    if samples == []:
      break
    
    # Choose a variable and count how many times it appears positive and negative in sampled solutions
    var = random.choice([v for v in I.getAssignment().keys() if v not in I.getFixedVars()])

    num_true = 0
    num_false = 0

    for sample in samples:
      if sample.get(var):
        num_true += 1
      else:
        num_false += 1
    
    # Set the variable and update the multiplier
    if num_true > num_false:
      I.modifyAssignment(var, True)
      multiplier *= len(samples) / num_true
    else:
      I.modifyAssignment(var, False)
      multiplier *= len(samples) / num_false

    # Simplify
    I.addFixedVar(var)
    unitPropagation(cnf, I)
    pureLiterals(cnf, I)
    
    n_left_vars = len([v for v in I.getAssignment().keys() if v not in I.getFixedVars()])

  return multiplier * exactCount(cnf, I)

## SampleCount

In [ ]:
# It implements SampleCount for the Model Counting problem.
def SampleCount(cnf, I, k, max_tries):
  lb = 2**len(I.getAssignment().keys())

  # Iterate for max_tries time to adjust the lower bound
  for j in range(max_tries):
    I.setFixedVars([])
    n_left_vars = len(I.getAssignment().keys())
    s = 0

    # While the formula is not feasible for ExactCount
    while n_left_vars > 15:
      s += 1
    
      # Sample at most k solutions (satisfying interpretations)
      i = 0
      samples = []
    
      while len(samples) < k and i < 2*k:
        if random.random() < 0.5:
          sol = WalkSAT(cnf, I, 0.25, 100, 100)
        else:
          sol = SASAT(cnf, I, 100, 1.0, 0.1, 0.05)

        if sol and I.getAssignment() not in samples:
          samples.append(I.getAssignment().copy())
        
        i += 1
      
      if samples == []:
        break

      # Choose a balanced variable (early stop when the balance is acceptable)
      left_vars = [v for v in I.getAssignment().keys() if v not in I.getFixedVars()]
      balanced_var = left_vars[0]
      balance = len(samples)
      num_true_bal = 0
      num_false_bal = 0

      for var in left_vars:
        num_true = 0
        num_false = 0

        for sample in samples:
          if sample.get(var):
            num_true += 1
          else:
            num_false += 1
        
        curr_balance = abs(num_true - num_false)

        if curr_balance <= balance:
          num_true_bal = num_true
          num_false_bal = num_false
          balance = curr_balance
          balanced_var = var
        
        if balance / len(samples) < 0.1:
          break
      
      # Set the chosen variable's assignment
      if num_true_bal == 0:
        I.modifyAssignment(balanced_var, False)
      elif num_false_bal == 0:
        I.modifyAssignment(balanced_var, True)
      elif random.random() < 0.5:
        I.flipAssignment(balanced_var)

      # Simplify
      I.addFixedVar(balanced_var)
      unitPropagation(cnf, I)
      pureLiterals(cnf, I)

      n_left_vars = len([v for v in I.getAssignment().keys() if v not in I.getFixedVars()])
    
    # Compute the approximated count and adjust the lower bound
    count = 2**s * exactCount(cnf, I)

    if count < lb:
      lb = count

  return lb

# Test

## Test

In [ ]:
# Try input:
# formula=random, n_vars=100, n_clauses=400, k_lit_in_C=3, k_exact=yes, sls=w, max_restart=100, max_flips=100, prob=0.25
def test():
  task = input("Task (enter 'sat', 'max', 'count'):\n")

  if task != 'max' and task != 'count':
    task = 'sat'

  if task == 'sat' or task == 'max':
    string = input("Enter the formula (enter 'random' if you want a random generated formula):\n")

    if string == 'random':
      n_vars = input("Enter the number of variables:\n")
      n_clauses = input("Enter the number of clauses:\n")
      k_lit_in_C = input("Enter the number k of literals in each clause:\n")
      k_exact = input("Enter 'yes' if k is exact, 'no' otherwise:\n")
      k_exact = False if k_exact == 'no' else True

      cnf, I = genRandomInstances(int(n_vars), int(n_clauses), int(k_lit_in_C), k_exact)
    else:
      formula, I = infixToPrefix(string)
      cnf, _ = convertToCNF(formula, 0)

      print('Formula in prefix notation:')
      print(formula)
      print("---------")

  else:
    cnf, I = genRandomInstances(20, 80, 3, True)

  print("Formula in Conjunctive Normal Form")
  cnf.printCNF()
  print("---------")

  if task == 'sat':
    unitPropagation(cnf, I)
    print("Assignment after Unit Propagation:")
    print(I.getAssignment())

    if isFormulaSatisfied(cnf, I):
      print("Model found!")
      return cnf, I
    print("---------")

    pureLiterals(cnf, I)
    print("Assignment after Pure Literals Rule:")
    print(I.getAssignment())

    if isFormulaSatisfied(cnf, I):
      print("Model found!")
      return cnf, I
    print("---------")

    algo = input("Enter 'g' for GSAT, 'w' for WalkSAT, 's' for SASAT and 'gen' for GLS:\n")
    max_restart = input("Enter max_restart:\n")

    if algo == 'g':
      max_flips = input("Enter max_flips:\n")
      res = GSAT(cnf, I, int(max_restart), int(max_flips))
    elif algo == 's':
      max_temp = input("Enter max_temp:\n")
      min_temp = input("Enter min_temp:\n")
      decay_rate = input("Enter decay_rate:\n")
      res = SASAT(cnf, I, int(max_restart), float(max_temp), float(min_temp), float(decay_rate))
    elif algo == 'gen':
      num_pop = input("Enter num_pop:\n")
      mutation_prob = input("Enter mutation_prob:\n")
      elitism_prop = input("Enter elitism_prop:\n")
      res = GLS(cnf, I, int(num_pop), int(max_restart), float(mutation_prob), float(elitism_prop))
    else:
      max_flips = input("Enter max_flips:\n")
      prob = input("Enter the probability value of random walk:\n")
      res = WalkSAT(cnf, I, float(prob), int(max_restart), int(max_flips))

    if res:
      print("Model found!")
    else:
      print("Model not found!")
    print('Final assignment:')
    print(I.getAssignment())

  elif task == 'max':
    algo = input("Enter 'j' for Johnson, 'b' for BBR and 'l' for Look-Ahead:\n")

    if algo == 'b':
      BBR(cnf, I)
      print("Number of satisfied clauses: {}".format(numberSatisfiedClauses(cnf, I)))
    elif algo == 'l':
      max_restart = input("Enter max_restart:\n")
      max_flips = input("Enter max_flips:\n")
      num_first_lvl = input("Enter num_first_lvl_vars:\n")
      num_sat = LookAhead(cnf, I, int(max_restart), int(max_flips), int(num_first_lvl))
      print("Number of satisfied clauses: {}".format(num_sat))
    else:
      Johnson(cnf, I)
      print("Number of satisfied clauses: {}".format(numberSatisfiedClauses(cnf, I)))
    
  else:
    algo = input("Enter 'a' for ApproxCount and 's' for SampleCount:\n")
    k = input("Enter the number of samples:\n")

    ec = exactCount(cnf, I)

    while ec < 10:
      cnf, I = genRandomInstances(20, 80, 3, True)
      ec = exactCount(cnf, I)

    if algo == 's':
      max_tries = input("Enter the number of max_tries:\n")
      ac = SampleCount(cnf, I, int(k), int(max_tries))
    else:
      ac = ApproxCount(cnf, I, int(k))
    
    print("Correct result: {}".format(ec))
    print("Approximated result: {}".format(ac))

test()